# Goal: Try to determine if a patient will die based on the data using SVM

#### Things I don't know:
<ol>
    <li> What does every value other than 1 and 2 on the classification_final data mean?</li>

## Type of data mining problem : Classification

In [1]:
!pip install keras
!pip install -q -U keras-tuner

In [94]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD,Adam
import tensorflow as tf
import pandas as pd

In [95]:
covid = pd.read_csv("Covid Data.csv")

In [96]:
print(covid.shape)

(1048575, 21)


In [97]:
covid.nunique(axis=0)

USMER                     2
MEDICAL_UNIT             13
SEX                       2
PATIENT_TYPE              2
DATE_DIED               401
INTUBED                   4
PNEUMONIA                 3
AGE                     121
PREGNANT                  4
DIABETES                  3
COPD                      3
ASTHMA                    3
INMSUPR                   3
HIPERTENSION              3
OTHER_DISEASE             3
CARDIOVASCULAR            3
OBESITY                   3
RENAL_CHRONIC             3
TOBACCO                   3
CLASIFFICATION_FINAL      7
ICU                       4
dtype: int64

In [98]:
covid

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,9999-99-99,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,9999-99-99,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,9999-99-99,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,9999-99-99,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97


## Determine if the patient died or not as the classification system

In [99]:
covid['DATE_DIED'] = covid['DATE_DIED'].apply(lambda x: 1 if x != '9999-99-99' else 0)

In [100]:
covid

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,1,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,1,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,1,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,1,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,1,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,0,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,0,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,0,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,0,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97


In [101]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype
---  ------                --------------    -----
 0   USMER                 1048575 non-null  int64
 1   MEDICAL_UNIT          1048575 non-null  int64
 2   SEX                   1048575 non-null  int64
 3   PATIENT_TYPE          1048575 non-null  int64
 4   DATE_DIED             1048575 non-null  int64
 5   INTUBED               1048575 non-null  int64
 6   PNEUMONIA             1048575 non-null  int64
 7   AGE                   1048575 non-null  int64
 8   PREGNANT              1048575 non-null  int64
 9   DIABETES              1048575 non-null  int64
 10  COPD                  1048575 non-null  int64
 11  ASTHMA                1048575 non-null  int64
 12  INMSUPR               1048575 non-null  int64
 13  HIPERTENSION          1048575 non-null  int64
 14  OTHER_DISEASE         1048575 non-null  int64
 15  CARDIOVASCULAR 

#### Basically no correlation with tobacco or asthma???

In [102]:
c = covid.corr().abs()
s = c['DATE_DIED']
so = s.sort_values(kind="quicksort")
so

HIPERTENSION            0.016940
DIABETES                0.018588
PNEUMONIA               0.026511
OBESITY                 0.029479
RENAL_CHRONIC           0.029907
INMSUPR                 0.031415
COPD                    0.031488
TOBACCO                 0.031958
CARDIOVASCULAR          0.032237
OTHER_DISEASE           0.033696
ASTHMA                  0.033924
PREGNANT                0.079864
SEX                     0.081383
USMER                   0.112671
MEDICAL_UNIT            0.149030
CLASIFFICATION_FINAL    0.196085
AGE                     0.320801
ICU                     0.508414
INTUBED                 0.509464
PATIENT_TYPE            0.515582
DATE_DIED               1.000000
Name: DATE_DIED, dtype: float64

In [103]:
from sklearn.model_selection import train_test_split

In [104]:
covid['DATE_DIED']

0          1
1          1
2          1
3          1
4          1
          ..
1048570    0
1048571    0
1048572    0
1048573    0
1048574    0
Name: DATE_DIED, Length: 1048575, dtype: int64

In [105]:
without_classification = covid.drop('DATE_DIED',axis = 1)
train_x, val_x, train_y, val_y = train_test_split(without_classification,covid.loc[:,"DATE_DIED"] , test_size=.2, shuffle=True)

## SVM Training

In [106]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,BatchNormalization,Input
from tensorflow.keras.optimizers import Adam,RMSprop
from sklearn import metrics
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers.experimental import RandomFourierFeatures

In [107]:
train_x.shape

(838860, 20)

In [108]:
val_x.shape

(209715, 20)

In [109]:
train_y.shape

(838860,)

### My own attempt at a SVM

In [110]:
model = Sequential()
model.add(Dense(128,activation='relu',input_shape=(20,)))
model.add(Dropout(0.25))
model.add(Dense(100, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='softmax'))

In [93]:
Earlystop = EarlyStopping(monitor='val_loss',mode='min',patience=15,min_delta=.001)
model.compile(optimizer='Adam',loss = 'binary_crossentropy',metrics=['accuracy'])
model.fit(train_x,train_y,validation_data=(val_x,val_y),epochs=50,batch_size=64,verbose=1)

Epoch 1/100
13108/13108 [==============================] - 21s 2ms/step - loss: 1.6177 - accuracy: 0.2560 - val_loss: 1.1873 - val_accuracy: 0.3895
Epoch 2/100
 2599/13108 [====>.........................] - ETA: 14s - loss: 1.1574 - accuracy: 0.6275

KeyboardInterrupt: 

### True SVM imported from Sklearn

In [114]:
from sklearn.svm import SVC
svm_model = SVC(kernel = 'sigmoid', degree = 5)
svm_model.fit(train_x,train_y)

SVC(degree=5, kernel='sigmoid')

In [115]:
prd_label = svm_model.predict(val_x)

## Accuracy score of 91.94%

In [118]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y,prd_label)

0.9194144434112963

### Testing to see what different degrees do to the accuracy (could lead to overfit)

In [ ]:
svm_model2 = SVC(kernel = 'sigmoid', degree = 7)
svm_model2.fit(train_x,train_y)
accuracy_score(val_y,svm_model2.predict(val_x))

### Test to see different C values